Ok so we have microegg variations

- hashmap vs indexmap
- aegraph unode but what about the linked list version of eclasses
- ac autoflattening _ built in AC matching
- extraction via top down memoization



There's a blog post to be had in "tabled memoization". On some level it's trivial because this is how you do DFS in a graph. You have to remember what you're seen or you're screwed.
DFS in dag you memoize to avoid redundancy work.
CF rational trees



In [ ]:
def extract_td(e, id):
    memo = {}
    # memo with None vs seen/open_node memo pair
    def worker(id):
        if id in memo:
            res = memo[id]
            # just return res
            if res is None:
                return None
            else:
                return res
        else:
            memo[id] = None # this branch is open
            #bestcost = None
            #for node in e.eclasses(id):
            #    args = map(worker, node.args)
            #    cost = 1 + cost(args)
            return min([App(node.f, map(worker,children))) for node in e.classes(id)], key=lambda x: cost(x))
    return worker(id)

        


# 
Moved to ac rpo

# rust


In [ ]:
use std::collections::HashMap;
use std::rc::Rc;
pub type Id = usize;
pub type Name = Rc<str>;

// The basics
#[derive(PartialEq, Eq, Hash, Clone, Debug)]
pub struct Node {
    f: Name,
    args: Vec<Id>,
}

#[derive(Default)]
pub struct EGraph {
    memo: HashMap<Node, Id>,
    uf: Vec<Id>,
}

impl EGraph {
    pub fn add_node(&mut self, node: Node) -> Id {
        let node = self.canonicalize_node(&node);
        if let Some(&id) = self.memo.get(&node) {
            id
        } else {
            let new_id = self.uf.len();
            self.uf.push(new_id);
            self.memo.insert(node, new_id);
            new_id
        }
    }

    pub fn add(&mut self, f: impl Into<Name>, args: impl Into<Vec<Id>>) -> Id {
        let node = Node {
            f: f.into(),
            args: args.into(),
        };
        self.add_node(node)
    }

    pub fn union(&mut self, a: Id, b: Id) {
        let a = self.find(a);
        let b = self.find(b);
        if a != b {
            self.uf[a] = b;
        }
    }

    pub fn find(&self, mut a: Id) -> Id {
        while self.uf[a] != a {
            a = self.uf[a];
        }
        a
    }

    pub fn is_eq(&self, a: Id, b: Id) -> bool {
        self.find(a) == self.find(b)
    }

    pub fn nodes_in_class(&self, class: Id) -> impl Iterator<Item = &Node> {
        self.memo
            .iter()
            .filter(move |(_, id)| self.is_eq(**id, class))
            .map(|(node, _)| node)
    }

    pub fn canonicalize_node(&self, node: &Node) -> Node {
        Node {
            f: node.f.clone(),
            args: node.args.iter().map(|id| self.find(*id)).collect(),
        }
    }

    pub fn rebuild(&mut self) {
        // copy needed for borrowing
        let nodes_copy = self.memo.clone();

        let mut keep_going = true;
        while keep_going {
            keep_going = false;
            for (node, id) in &nodes_copy {
                let new_node = self.canonicalize_node(node);
                let new_id = self.add_node(new_node);
                if !self.is_eq(new_id, *id) {
                    self.union(new_id, *id);
                    keep_going = true;
                }
            }
        }
        println!("rebuilt!")
    }
}

// e-matching

#[derive(PartialEq, Eq, Hash, Clone, Debug)]
pub enum Pattern {
    Var(Name),
    App(Name, Vec<Pattern>),
}

pub type Subst = HashMap<Name, Id>;

impl EGraph {
    pub fn ematch(&self, pat: &Pattern, class: Id) -> Vec<Subst> {
        self.ematch_rec(0, pat, class, Default::default())
    }
    pub fn ematch_rec(
        &self,
        depth: usize,
        pat: &Pattern,
        class: Id,
        mut subst: Subst,
    ) -> Vec<Subst> {
        println!("{:d$}subst: {subst:?}", "", d = depth * 2,);
        println!("{:d$}matching {pat:?} at {class:?}", "", d = depth * 2,);
        match pat {
            Pattern::Var(name) => {
                if let Some(&id) = subst.get(name) {
                    if self.is_eq(id, class) {
                        return vec![subst];
                    } else {
                        return vec![];
                    }
                } else {
                    subst.insert(name.clone(), class);
                    return vec![subst];
                }
            }
            Pattern::App(f, args) => {
                let mut results = vec![];
                for node in self.nodes_in_class(class) {
                    let mut todo = vec![subst.clone()];
                    println!(
                        "{:d$}matching {pat:?} at {class:?} - {node:?}",
                        " ",
                        d = depth * 2,
                    );
                    if node.f == *f && node.args.len() == args.len() {
                        for (pa, na) in args.iter().zip(node.args.iter()) {
                            todo = todo
                                .into_iter()
                                .flat_map(|subst| self.ematch_rec(depth + 1, pa, *na, subst))
                                .collect();
                        }
                    }
                    results.extend(todo);
                }
                results
            }
        }
    }
}


In [ ]:

let mut eg = EGraph::default();
let a = eg.add("a", []);
let b = eg.add("b", []);
let c = eg.add("c", []);
let f1 = eg.add("f", [a, b]);
let f2 = eg.add("f", [a, c]);

eg.union(b, c);
assert!(!eg.is_eq(f1, f2));

eg.rebuild();
assert!(eg.is_eq(f1, f2));




rebuilt!


In [ ]:
fn test_match() {
    let mut eg = EGraph::default();
    let a = eg.add("a", []);
    let b = eg.add("b", []);

    let f1 = eg.add("f", [a, a]);
    let f2 = eg.add("f", [b, b]);
    let f3 = eg.add("f", [a, b]);

    eg.union(f1, f2);
    eg.union(f2, f3);
    eg.rebuild();

    use Pattern::*;
    let pat_fxx = App("f".into(), vec![Var("x".into()), Var("x".into())]);
    let pat_fxy = App("f".into(), vec![Var("x".into()), Var("y".into())]);

    let fxx_matches = eg.ematch(&pat_fxx, f1);
    assert_eq!(fxx_matches.len(), 2);

    let fxy_matches = eg.ematch(&pat_fxy, f1);
    assert_eq!(fxy_matches.len(), 3);
}

# Rudi
https://egraphs.zulipchat.com/#narrow/channel/556617-rambling/topic/Rudi.20Schneider/near/574808445

Rudi syas flatterns

In [3]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Term:
    f: str
    args: tuple["Term"]

    def size(self) -> int:
        return 1 + sum(map(Term.size, self.args))

    # ground knuth-bendix comparison. 
    def __lt__(x: "Term", y: "Term") -> bool:
        # We can actually use lexicographical list comparison of flatterms here.
        return x.to_flatterm() < y.to_flatterm()

    def to_flatterm(self):
        l = [self.size(), self.f]
        for c in self.args:
            l.extend(c.to_flatterm())
        return l

    def __add__(self, other):
        return Term("+", (self, other))

    def __repr__(self):
        if len(self.args) > 0:
            return self.f + "(" + ", ".join(map(str, self.args)) + ")"
        else:
            return self.f

class GKB:
    def __init__(self):
        self.canon = {}

    def find_children(self, x: Term) -> Term:
        return Term(x.f, tuple(map(self.find, x.args)))

    def find(self, x: Term) -> Term:
        x = self.find_children(x)
        if x in self.canon:
            x = self.canon[x]
        return x

    def union(self, x: Term, y: Term):
        x = self.find(x)
        y = self.find(y)

        if x < y:
            self.canon[y] = x
        elif x > y:
            self.canon[x] = y
        else:
            assert(x == y)
            return

    def rebuild(self):
        while True:
            new = GKB()
            for (s, t) in self.canon.items():
                s = self.find_children(s) # This corresponds to computing a canonical e-node
                t = self.find(t)
                new.union(s, t)
            if self.canon == new.canon:
                break
            else:
                self.canon = new.canon

    def is_equal(self, x: Term, y: Term) -> bool:
        return self.find(x) == self.find(y)

gkb = GKB()

a = Term("a", ())
b = Term("b", ())
c = Term("c", ())
d = Term("d", ())
gkb.union(a, c); gkb.rebuild()
gkb.union(b, d); gkb.rebuild()
gkb.union(a, c+c); gkb.rebuild()

print(gkb.is_equal(a+(b+d), c+(d+b)))
print(gkb.canon)

True
{c: a, d: b, +(a, a): a}


In [ ]:
["b"] > ["a", "b"] > ["a", "a", "b"] > ["a", "a", "a", "b"] # ...

True

In [ ]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Node:
    f: str
    args: tuple["Id"]

    def __str__(self):
        if self.args == ():
            return self.f
        return self.f + "(" + ", ".join(map(str, self.args)) + ")"


# We use ptrs as Ids.
class Id: pass

class EGraph:
    def __init__(self):
        # combines hashcons & unionfind into one thing.
        # the hashcons_uf takes in a "thing" and returns a "more canonical thing".
        self.hashcons_uf = {}
    
    # combines find & lookup & add & add_expr into one thing.
    def find_add_expr(self, x):
        # This part comes from add/lookup
        if isinstance(x, Node):
            x = Node(x.f, tuple([self.find_add_expr(a) for a in x.args]))

        # This part comes from find.
        if x in self.hashcons_uf:
            x = self.hashcons_uf[x]

        if isinstance(x, Node):
            # This comes from add
            i = Id()
            self.hashcons_uf[x] = i
            return i
        else:
            return x

    def union(self, x, y):
        x = self.find_add_expr(x)
        y = self.find_add_expr(y)
        if x == y: return

        self.hashcons_uf[x] = y
        self.rebuild_path_compress()

    # combines rebuilding & path compression
    def rebuild_path_compress(self):
        for (k, v) in list(self.hashcons_uf.items()):
            # comes from rebuild
            if isinstance(k, Node):
                k = Node(k.f, tuple([self.find_add_expr(a) for a in k.args]))
            vv = self.find_add_expr(v)
            if k in self.hashcons_uf:
                self.union(vv, self.hashcons_uf[k])
            else:
                self.hashcons_uf[k] = vv

    def is_equal(self, x, y):
        x = self.find_add_expr(x)
        y = self.find_add_expr(y)
        return x == y

a = Node("a", ())
b = Node("b", ())
f = lambda x, y: Node("f", (x, y))

eg = EGraph()

i1 = eg.find_add_expr(f(a, b))
i2 = eg.find_add_expr(f(b, a))

eg.union(a, b)

print(eg.is_equal(i1, i2))